# PyCity School Analysis

In [308]:

# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# # Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load,  index_col = False)
student_data = pd.read_csv(student_data_to_load,  index_col = False)



In [309]:
renamed_school_data = school_data.rename(columns={"name": "School Name", "type" : "School Type"})
renamed_student_data = student_data.rename(columns= {"school": "School Name"})

In [310]:
# # Combine the data into a single dataset
school_data_complete = pd.merge(renamed_student_data, renamed_school_data, how="left", on=["School Name", "School Name"])

# District Summary

In [311]:
# District Summary
# Number of schools can be infered by the school data's count() or the len()
Total_Schools = len(school_data) 

# Number of students can be infered by the students len()
Total_Students = len(student_data)

# Total budget for all the schools is the sum of budgets of individual schools
Total_Budget = school_data["budget"].sum()

# Average of  maths  is the  mean() of all the students in maths
Average_Math_Score = student_data["math_score"].mean()

# Average of  reading score  is the  mean() of all the students in reading
Average_Reading_Score = student_data["reading_score"].mean()

# % of students passing maths i.e., getting over 70 (please note the usage of query to filter records)
Percent_Students_Passing_Maths = len(student_data.query('math_score>70')) * 100 / Total_Students

# % of students passing reading i.e., getting over 70 (please note the usage of query to filter records)
Percent_Students_Passing_Reading = len(student_data.query('reading_score>70')) * 100 / Total_Students

# Overall passing is the average of math and reading (formula provided to us) 
Overall_Passing_Rate = ( Average_Math_Score + Average_Reading_Score )/ 2

# Populating the dictionary with all the computed values above 
District_Summary = [{  "Total Schools":Total_Schools
                    , "Total Students": '{:,}'.format(Total_Students)
                    , "Total Budget": '${:,.2f}'.format(Total_Budget)
                    , "Average Math Score": Average_Math_Score
                    , "Average Reading Score" : Average_Reading_Score
                    , "% Passing Math": Percent_Students_Passing_Maths
                    , "% Passing Reading": Percent_Students_Passing_Reading
                    , "% Overall Passing Rate": Overall_Passing_Rate
                   }]

# Convert to Dataframe to display tabular format
District_Summary_Dataframe = pd.DataFrame(District_Summary)

# Rearranged the columns
District_Summary_Dataframe = District_Summary_Dataframe[["Total Schools"
                                                         ,"Total Students"
                                                         , "Total Budget"
                                                         ,"Average Math Score"
                                                         , "Average Reading Score"
                                                         , "% Passing Math"
                                                         , "% Passing Reading"
                                                         , "% Overall Passing Rate"
                                                        ]]
District_Summary_Dataframe

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,72.392137,82.971662,80.431606


## School Summary

In [312]:
# Grouped the data on School Name and aggregated columns that needed to be displayed
School_Summary = school_data_complete.groupby(["School Name"]).agg({'math_score': 'mean'
                                                                    ,'reading_score':'mean'
                                                                    ,'size':'size'
                                                                    ,'budget':'min'
                                                                    ,'School Type':'min'
                                                                    ,'School Name': 'min'
                                                                   })
# Computed the Per student budget 
School_Summary['Per Student Budget'] = School_Summary['budget'] / School_Summary['size']
# School_Summary

In [313]:
# Computed the number of students passed in reading (getting > 70)
School_Summary_Pass_Reading = school_data_complete.query('reading_score>70').groupby(["School Name"]).agg({'reading_score':'size'
                                                                                                           ,'School Name':'min' })

# Renamed the column
School_Summary_Pass_Reading = School_Summary_Pass_Reading.rename(columns={"reading_score": "Passed Reading"})
# School_Summary_Pass_Reading

In [314]:
# Computed the number of students passed in maths (getting > 70)
School_Summary_Pass_Math = school_data_complete.query('math_score>70').groupby(["School Name"]).agg({'math_score':'size'
                                                                                                    , 'School Name':'min'})

# Renamed the column
School_Summary_Pass_Math = School_Summary_Pass_Math.rename(columns={"math_score": "Passed Math"})
# School_Summary_Pass_Math

In [315]:
# Merge the scores students passing maths and reading 
School_Pass_Math_And_Reading = pd.merge(School_Summary_Pass_Reading, School_Summary_Pass_Math, how="inner", on=["School Name", "School Name"])
# School_Pass_Math_And_Reading

/Users/ravindramohan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'School Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [316]:
# Merge the passing students with summary i.e., percentages 
School_Overall = pd.merge(School_Summary, School_Pass_Math_And_Reading, how="left", on=["School Name", "School Name"])

# Renamed the columns
School_Overall = School_Overall.rename(columns={"math_score": "Average Math Score"
                               , "reading_score":"Average Reading Score" 
                               ,"budget":"Total School Budget"
                               ,"size":"Total Students"
                              })
# Computed the percentages
School_Overall['% Passing Math'] = School_Overall['Passed Math'] * 100/ School_Overall['Total Students']
School_Overall['% Passing Reading'] = School_Overall['Passed Reading'] * 100 / School_Overall['Total Students']
School_Overall['% Overall Passing Rate'] = (School_Overall['% Passing Math'] + School_Overall['% Passing Reading']) / 2

# Removed extra columns
del School_Overall['Passed Math']
del School_Overall['Passed Reading']

# Rearranged the columns as per the requirements 
School_Overall = School_Overall[["School Name"
                                 ,"School Type"
                                 ,"Total Students"
                                 , "Total School Budget"
                                 ,"Per Student Budget"
                                 ,"Average Math Score"
                                 ,"Average Reading Score"
                                 ,"% Passing Math"
                                 ,"% Passing Reading"
                                 ,"% Overall Passing Rate"
                                ]] 
# School_Overall = School_Overall.groupby(["School Name"])
# School_Overall

/Users/ravindramohan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'School Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


## Top Performing Schools (By Passing Rate)

In [321]:
# Sorted and displayed top 5
School_Overall_Top5 = School_Overall.sort_values(by=['% Overall Passing Rate'], ascending=False)
School_Overall_Top5.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,90.932983,93.254490,92.093736
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,91.683992,92.203742,91.943867
14,Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,90.277778,93.444444,91.861111
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,89.558665,93.864370,91.711518
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,90.632319,92.740047,91.686183


## Bottom Performing Schools (By Passing Rate)

In [303]:
# Sorted and displayed bottom 5
School_Overall_Bottom5 = School_Overall.sort_values(by=['% Overall Passing Rate'] , ascending=True) 
School_Overall_Bottom5.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,64.066017,77.744436,70.905226
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,63.318478,78.813850,71.066164
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,63.852132,78.281874,71.067003
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,63.750424,78.433367,71.091896
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,64.746494,78.187702,71.467098


## Math Scores by Grade

In [304]:
#Stripped 'th' from the grade and converted it to int, to sort on it
renamed_student_data["grade"]  = renamed_student_data["grade"].replace('th','',regex=True).astype('int')

# Grouped on school name and then on grade. Unstock i.e., unpivot to convert rows into columns
Renamed_Student_Data_Math = renamed_student_data.groupby(['School Name','grade']).agg({'School Name':'min'
                                                                                       ,'math_score':'mean' }).unstack()
# Removing the extra columns that are not needed to be displayed
Renamed_Student_Data_Math = Renamed_Student_Data_Math[["math_score"]]

# Suppressed the display of 'math_score' header for a cleaner formatting
Renamed_Student_Data_Math = Renamed_Student_Data_Math.xs('math_score', axis=1, drop_level=True)

# Renamed the grade columns by adding 'th' 
Renamed_Student_Data_Math.set_axis(['9th','10th','11th','12th'], axis="columns",inplace=True)
Renamed_Student_Data_Math

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [208]:
# Grouped on school name and then on grade. Unstock i.e., unpivot to convert rows into columns
Renamed_Student_Data_Reading_Score = renamed_student_data.groupby(['School Name','grade']).agg({'School Name':'min','reading_score':'mean' }).unstack()

# Removing the extra columns that are not needed to be displayed
Renamed_Student_Data_Reading_Score =  Renamed_Student_Data_Reading_Score[["reading_score"]]

# Suppressed the display of 'reading_score' header for a cleaner formatting
Renamed_Student_Data_Reading_Score = Renamed_Student_Data_Reading_Score.xs('reading_score', axis=1, drop_level=True)

# Renamed the grade columns by adding 'th' 
Renamed_Student_Data_Reading_Score.set_axis(['9th','10th','11th','12th'], axis="columns",inplace=True)

Renamed_Student_Data_Reading_Score

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [322]:
# Bin 
spending_bins = [0, 585, 615, 645, 675]

# columns for aggregation
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# store the dataset in a local variable
School_Overall_Spending = School_Overall

# Bin values into descrite intervals
School_Overall_Spending["Spending Ranges (Per Student)"] = pd.cut(School_Overall_Spending["Per Student Budget"], spending_bins, labels=group_names)

# select columns by its index positions 
School_Overall_Spending = School_Overall_Spending.iloc[:,5:11]

# Aggregate on Average Math scores
School_Overall_Spending = School_Overall_Spending.groupby(['Spending Ranges (Per Student)']).agg({'Average Math Score':'mean'
                                                                                                  ,'Average Reading Score':'mean'
                                                                                                  ,'% Passing Math':'mean'
                                                                                                  ,'% Passing Reading':'mean'
                                                                                                  ,'% Overall Passing Rate':'mean'})
School_Overall_Spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,90.350436,93.325838,91.838137
$585-615,83.599686,83.885211,90.788049,92.410786,91.599418
$615-645,79.079225,81.891436,73.021426,83.214343,78.117884
$645-675,76.997210,81.027843,63.972368,78.427809,71.200088


## Scores by School Size

In [287]:
#Bins intervals
size_bins = [0, 1000, 2000, 5000]

# columns for aggregation
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# store the dataset in a local variable
School_Overall_SchoolSize = School_Overall

# Bin values into descrite intervals
School_Overall_SchoolSize["School Size"] = pd.cut(School_Overall_SchoolSize["Total Students"], size_bins, labels=group_names)

# select columns by its index positions 
School_Overall_SchoolSize = School_Overall_SchoolSize.iloc[:,5:11]

# Aggregate on Average Reading scores
School_Overall_SchoolSize = School_Overall_SchoolSize.groupby(['School Size']).agg({'Average Math Score':'mean','Average Reading Score':'mean','% Passing Math':'mean','% Passing Reading':'mean','% Overall Passing Rate':'mean'})
School_Overall_SchoolSize

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067


## Scores by School Type

In [249]:
# Aggregate on School type 
School_Type_Scores= School_Overall.groupby(['School Type']).agg({'Average Math Score':'mean'
                                                                 ,'Average Reading Score':'mean'
                                                                 ,'% Passing Math':'mean'
                                                                 ,'% Passing Reading':'mean'
                                                                 ,'% Overall Passing Rate':'mean'})
School_Type_Scores

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
